```
References
under review.

Copyright (c) 2022 Gyorgy Denes (gdenes355@gmail.com), Pontus Andersson, 
Tomas Akenine-Möller, Kalle Åström, Magnus Oskarsson, William H. McIlhagga

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:
The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
```

This reference implementation is based on the original Matlab code. 
In the paper, the application results are based on the Matlab implementation.

In [8]:
import numpy as np
import cv2 as cv
from enum import Enum

In [2]:
class FilterType(Enum):
    T_ALL = 0  # final filter from the paper
    T_LOW = 1  # filter from low-contrast cases only
    T_ALL_INTERP = 2  # disef function fitted to all 60Hz data (t_all), interpolated
    T_LOW_INTERP = 3  # disef function fitted to t_low, interpolated
    D_ALL = 4  # analytical disef function fitted to t_all, sampled
    D_LOW = 5  # analytical disef function fitted to t_low, sampled

def get_temporal_edge_filter(t, ffilter: FilterType = FilterType.D_ALL):
    """ 
    Return the temporal edge filter from the paper.

    Parameters
    ----------
    
    t : np.ArrayLike<float32>
        time points to sample the filter at

    ffilter : FilterType
        type of filter to use. Defaults to d_all
    
    """
    if ffilter == FilterType.D_ALL:
        s_in_sec = 1.74443648427644 / 60  # s value from paper
        t0 = np.mean(t)
        t = t - t0
        filt = -np.sign(t) * np.exp(-np.abs(t)/s_in_sec)
    elif ffilter == FilterType.D_LOW:
        s_in_sec = 1.2386001106974815 / 60  # s value from paper
        t0 = np.mean(t)
        t = t - t0
        filt = -np.sign(t) * np.exp(-np.abs(t)/s_in_sec)
    else:
        if ffilter == FilterType.T_ALL:
            samples = np.array([3.955975174903869629e-01, -1.020166635513305664e+00, -3.440489530563354492e+00, -5.091692924499511719e+00, -5.026970863342285156e+00, -2.602464199066162109e+00, 3.469745635986328125e+00, 1.378295803070068359e+01, 2.633740043640136719e+01, 3.921465301513671875e+01, 1.188159108161926270e+00, -2.439352607727050781e+01, -1.657835006713867188e+01, -9.859210014343261719e+00, -6.638015270233154297e+00, -6.391635417938232422e+00, -5.401230812072753906e+00, -3.818691492080688477e+00, -1.050134897232055664e+00, 2.127528429031372070e+00, 2.900552988052368164e+00], dtype=np.float32)
        elif ffilter == FilterType.T_LOW:
            samples = np.array([1.909064141608989573e-01, 3.386727748829402862e-01, 6.008140111533419869e-01, 1.065859150098313224e+00, 1.890860909963615333e+00, 3.354434758568847652e+00, 5.950851535511070267e+00, 1.055696012785268678e+01, 1.872831249040729062e+01, 3.322449687130618656e+01, -0.000000000000000000e+00, -3.322449687130618656e+01, -1.872831249040729062e+01, -1.055696012785268678e+01, -5.950851535511070267e+00, -3.354434758568847652e+00, -1.890860909963615333e+00, -1.065859150098313224e+00, -6.008140111533419869e-01, -3.386727748829402862e-01, -1.909064141608989573e-01], dtype=np.float32)
        elif ffilter == FilterType.T_ALL_INTERP:
            samples = np.array([2.467014074325561523e+00, 1.195966601371765137e+00, -1.652753710746765137e+00, -5.535111904144287109e+00, -8.609863281250000000e+00, -8.827475547790527344e+00, -4.716462135314941406e+00, 1.016245746612548828e+01, 3.350759124755859375e+01, 5.394515991210937500e+01, 6.111305236816406250e+00, -3.482356262207031250e+01, -1.925815010070800781e+01, -5.424099922180175781e+00, -1.625993967056274414e+00, -7.347380161285400391e+00, -7.372952938079833984e+00, -4.587455749511718750e+00, -2.869678020477294922e+00, 2.272292226552963257e-02, 2.123717546463012695e+00], dtype=np.float32)
        elif ffilter == FilterType.T_ALL_INTERP:
            samples = np.array([3.219703752655577594e-02, 7.218537057656362887e-02, 1.618387319323421891e-01, 3.628404889282670909e-01, 8.134840086410377591e-01, 1.823821355409757761e+00, 4.088985525364493334e+00, 9.167456328464750115e+00, 2.055332673911014041e+01, 4.608031114726507838e+01, -0.000000000000000000e+00, -4.608031114726507838e+01, -2.055332673911014041e+01, -9.167456328464750115e+00, -4.088985525364493334e+00, -1.823821355409757761e+00, -8.134840086410377591e-01, -3.628404889282670909e-01, -1.618387319323421891e-01, -7.218537057656362887e-02, -3.219703752655577594e-02], dtype=np.float32)
        else:
            raise ValueError("Filter not recognised")
        samplets = np.arange(0, samples.shape[0]) / 60
        filt = np.interp(t, samplets, samples)

    # remove nan values
    filt = filt[~np.isnan(filt)]
    t = t[~np.isnan(filt)]

    # normalise so +ves sum to 1, -ves sum to -1
    filt[filt > 0] = filt[filt>0] / np.sum(filt[filt>0])
    filt[filt < 0] = -filt[filt<0] / np.sum(filt[filt<0])

    return filt, t
    

In [ ]:
def predict_flicker_in_image(frame_a, frame_b, ppd: float = 52.0, fps: int = 120, ffilter: FilterType = FilterType.D_ALL, psyalpha: float = -0.1008, psybeta: float = 0.9061):
    """
    Predict the amount of flicker visible when displaying a pair of alternating frames

    Paramters
    ---------

    frame_a: np.ArrayLike<float32,float32,float32>
        frame in CIE XYZ

    frame_b: np.ArrayLike<float32,float32,float32>
        frame in CIE XYZ

    ppd: float = 52.0
        pixels per angular degrees. Defaults to 52 ppd

    fps: int = 120
        frames per second. Defaults to 120

    ffilter: FilterType = D_ALL
        temporal edge detection to use

    
    psyalpha: float
        alpha value of psychometric function. Default: best fitting reported in paper for D_ALL

    psybeta: float
        beta value of psychometric function. Default: best fitting reported in paper for D_ALL
    """
    assert(frame_a.shape == frame_b.shape)

    # create a 1.0 second video with 2 samples per frame
    SAMPLES_PER_FRAME = 2
    t = np.linspace(0, 1, fps * SAMPLES_PER_FRAME + 1)

    # video frames (luminance only)
    vid = np.zeros((t.shape[0], frame_a.shape[0], frame_a.shape[1]))
    allmask = np.arange(0, t.shape[0])
    amask = (allmask // SAMPLES_PER_FRAME) % 2 == 0
    bmask = (allmask // SAMPLES_PER_FRAME) % 2 == 1
    vid[amask,:,:] = frame_a[:,:,1]  # luminance is at channel 1
    vid[bmask,:,:] = frame_b[:,:,1]

    # filtering
    filt, tfilt = get_temporal_edge_filter(t[t < 0.9], ffilter)
    filtered = np.apply_along_axis(lambda m: np.convolve(m, filt, "valid"), axis=0, arr=vid)
    filtered = np.squeeze(np.max(np.abs(filtered), axis=0))
    filtered = cv.GaussianBlur(filtered, (0,0), sigmaX=int(0.36*ppd), sigmaY=int(0.36*ppd), borderType=cv.BORDER_REFLECT)

    P = 1 - np.exp(psyalpha * np.power(filtered, psybeta))
    return P


In [ ]:
from matplotlib import pyplot as plt 
def show_image(img):
    plt.figure(figsize=(6*img.shape[1] / img.shape[0], 6))
    plt.axis('off')
    if np.ndim(img) == 2:
        plt.imshow(img, cmap='gray', vmin=0, vmax=1)
    else:
        plt.imshow(img[:,:,:])


import h5py
IMAGE_NUM = 9
with h5py.File('stimuli.mat', 'r') as f:
    frame_as = np.array(f.get("stimuli").get("ref"))
    frame_a_XYZ = np.moveaxis(np.moveaxis(np.squeeze(frame_as[:,:,:,IMAGE_NUM]), 0, -1), 1, 0)
    frame_bs = np.array(f.get("stimuli").get("blur"))
    frame_b_XYZ = np.moveaxis(np.moveaxis(np.squeeze(frame_bs[:,:,:,IMAGE_NUM]), 0, -1), 1, 0)
    
P = predict_flicker_in_image(frame_a_XYZ, frame_b_XYZ, 52, 120)

show_image(P)